In [2]:
import pandas as pd
import numpy as np
import duckdb

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_folder = "staticNotDeployed/"
static_folder = "static/"

In [28]:
# scenario_BNZ_path = "simulations_new/BNZ.csv"
# scenario_holder_path = "simulations_new/BNZ.csv"

# New data march 2025
# scenario_BNZ_path = data_folder + "simulations_new/march2025/BNZ_£millions_annualy.csv"
# socio_factors_path = "UK_Archetypes_global_measures.csv"

# april 2025
# scenario_BNZ_path = data_folder + "simulations_new/april2025/cleaned_total_outputs_millions.csv"
# socio_factors_path = data_folder + "simulations_new/april2025/SEF.csv"

# may 2025
# scenario_BNZ_path = data_folder + "simulations_new/may2025/CB7_outputs_final_millions.csv"
socio_factors_path = data_folder + "simulations_new/april2025/SEF.csv"

# june 2025
scenario_BNZ_path = data_folder + "simulations_new/june2025/Model_run_outputs_hassle_cost_fix.csv"

## Create pandas tables

In [29]:
# df_socio = pd.read_csv(static_folder+socio_factors_path)
df_socio = pd.read_csv(socio_factors_path)

In [30]:
# Remove one row on NaN
df_socio = df_socio.dropna()

In [31]:
df_socio = df_socio.convert_dtypes()

In [32]:
df_socio

,Nation,Region,LAD,LSOA.DZ.CD,LSOA.DZ.NM,Households,Population,Under.35,Over.65,EPC,...,Rurality,House.value,Fuel.Type,Fuel.consumption.total,Floor.area,Gas.flag,Number.cars,Urban.trips,Total.vkm,Urban.vkm
0,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,249,738,0.45664,0.154472,4,...,0.0,137687.9444,3,37665.0,180.0,0,2,414.15324,12922.63505,2584.316217
1,NI,Northern Ireland,N09000001,N20000002,Dunsilly_B1,127,331,0.372372,0.204204,4,...,0.0,149950.0,3,35381.25,166.5,0,2,375.943839,11730.40451,2345.889557
2,NI,Northern Ireland,N09000001,N20000003,Dunsilly_A2,128,365,0.442623,0.229508,4,...,0.5,137687.9444,3,32461.0,143.0,0,2,368.684492,11503.89439,2300.591227
3,NI,Northern Ireland,N09000001,N20000004,Dunsilly_A3,219,663,0.470588,0.176471,5,...,0.5,137687.9444,3,30303.0,126.0,0,2,386.977279,12074.67589,2414.738218
4,NI,Northern Ireland,N09000001,N20000005,Dunsilly_B2,231,676,0.446746,0.189349,4,...,0.0,134250.0,3,35165.625,181.5,0,2,407.61788,12718.71519,2543.535571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46421,Scotland,Scotland,S12000040,S01013477,Broxburn South - 06,311,644,0.34,0.23,3,...,0.9,175000.0,1,31622.0,97.0,1,2,391.755872,15827.26805,10490.75574
46422,Scotland,Scotland,S12000040,S01013478,Broxburn East - 01,352,823,0.42,0.14,3,...,0.9,130000.0,1,12784.0,68.0,1,2,699.209847,15223.57237,10090.60937
46423,Scotland,Scotland,S12000040,S01013479,Broxburn East - 02,445,734,0.34,0.26,3,...,0.9,72500.0,1,15801.0,69.0,1,2,461.180963,14259.49696,9451.593232
46424,Scotland,Scotland,S12000040,S01013480,Broxburn East - 03,303,703,0.4,0.23,3,...,0.9,72500.0,1,17025.0,75.0,1,2,359.522793,13406.18716,8885.995642


In [33]:
df_socio.rename(columns=lambda x: x.replace('.', '_'), inplace=True)

In [34]:
df_socio.columns

Index(['Nation', 'Region', 'LAD', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Households',
       'Population', 'Under_35', 'Over_65', 'EPC', 'Median_Income', 'Tenure',
       'Typology', 'Unemployment', 'Rurality', 'House_value', 'Fuel_Type',
       'Fuel_consumption_total', 'Floor_area', 'Gas_flag', 'Number_cars',
       'Urban_trips', 'Total_vkm', 'Urban_vkm'],
      dtype='object')

In [35]:
# EPC and gas_flag have weird values
df_socio.dtypes

Nation                    string[python]
Region                    string[python]
LAD                       string[python]
LSOA_DZ_CD                string[python]
LSOA_DZ_NM                string[python]
Households                         Int64
Population                         Int64
Under_35                         Float64
Over_65                          Float64
EPC                               object
Median_Income                    Float64
Tenure                             Int64
Typology                           Int64
Unemployment                     Float64
Rurality                         Float64
House_value                      Float64
Fuel_Type                          Int64
Fuel_consumption_total           Float64
Floor_area                       Float64
Gas_flag                           Int64
Number_cars                        Int64
Urban_trips                      Float64
Total_vkm                        Float64
Urban_vkm                        Float64
dtype: object

In [36]:
set(df_socio.EPC)

{1, 2, '2', 3, '3', 4, '4', 5, '5', 6, '6', 7, '7', 'd', 'e'}

In [37]:
set(df_socio.Gas_flag)

{0, 1}

In [38]:
# Replace the string 'unknown' with pd.NA
df_socio['Gas_flag'] = df_socio['Gas_flag'].replace('Y', pd.NA)

# Step 2: Convert to numeric (converts '1.0' to 1.0 as float)
df_socio['Gas_flag'] = pd.to_numeric(df_socio['Gas_flag'], errors='coerce')

df_socio['Gas_flag'] = df_socio['Gas_flag'].astype('Int16')

In [39]:
# Replace the string 'unknown' with pd.NA
df_socio['EPC'] = df_socio['EPC'].replace('d', pd.NA)

# Step 2: Convert to numeric (converts '1.0' to 1.0 as float)
df_socio['EPC'] = pd.to_numeric(df_socio['EPC'], errors='coerce')

df_socio['EPC'] = df_socio['EPC'].astype('Int16')

In [40]:
set(df_socio.Region)

{'East Midlands',
 'Eastern England',
 'London',
 'North East',
 'North West',
 'Northern Ireland',
 'Scotland',
 'South East',
 'South West',
 'Wales',
 'West Midlands',
 'Yorks & Humber'}

In [41]:
# Split Eng/Wales into England and Wales in Nation column

In [42]:
df_socio.loc[df_socio.Region == "Wales", "Nation"] = "Wales"

In [43]:
df_socio.loc[(df_socio.Region != "Wales") & (df_socio.Nation == "Eng/Wales"), "Nation"] = "England"

In [44]:
set(df_socio.Nation)

{'England', 'NI', 'Scotland', 'Wales'}

#### Add LADs to the SEF table from the lookup LAD tables

In [45]:
# df_lad_NI = pd.read_csv("static/LAD/NI_DZ_LAD.csv")
df_lad_Eng = pd.read_csv("static/LAD/Eng_wales_LSOA_LADs.csv")
# df_lad_Scotland = pd.read_csv("static/LAD/Scotland_DZ_LA.csv", encoding='latin1')

In [46]:
df_lad_Eng.head()

,LSOA11CD,LSOA11NM,LSOA21CD,LSOA21NM,CHGIND,LAD22CD,LAD22NM,LAD22NMW,ObjectId
0,E01000001,City of London 001A,E01000001,City of London 001A,U,E09000001,City of London,NaN,1
1,E01000002,City of London 001B,E01000002,City of London 001B,U,E09000001,City of London,NaN,2
2,E01000003,City of London 001C,E01000003,City of London 001C,U,E09000001,City of London,NaN,3
3,E01000005,City of London 001E,E01000005,City of London 001E,U,E09000001,City of London,NaN,4
4,E01000006,Barking and Dagenham 016A,E01000006,Barking and Dagenham 016A,U,E09000002,Barking and Dagenham,NaN,5


In [47]:
# Some LADs in England were not matched correclty with the shapefiles:
# it works with the ones from the lookup files, but we need to change the values here

# Create a lookup dictionary
lookup = dict(zip(df_lad_Eng['LSOA11CD'], df_lad_Eng['LAD22CD']))
# lookup2 = dict(zip(df_lad_Eng['LSOA21CD'], df_lad_Eng['LAD22CD']))

# Loop through df and update status if there's a match
for idx, row in df_socio[df_socio.Nation == "England"].iterrows():
    lsoa = row['LSOA_DZ_CD']
    if lsoa in lookup:
        df_socio.at[idx, 'LAD'] = lookup[lsoa]

### Cobenefit table

In [48]:
df = pd.read_csv(scenario_BNZ_path)

In [49]:
# Step: Drop rows where any column contains the value '#DIV/0!'
# There are rows with these weird values in the new dataset
df = df[~df.isin(['#DIV/0!']).any(axis=1)]

In [50]:
# To not change the queries in the app
df["scenario"] = "BNZ"

In [52]:
# len(set(df.LAD))

In [53]:
df.dtypes

Lookup.Value     object
Coben            object
2025            float64
2026            float64
2027            float64
2028            float64
2029            float64
2030            float64
2031            float64
2032            float64
2033            float64
2034            float64
2035            float64
2036            float64
2037            float64
2038            float64
2039            float64
2040            float64
2041            float64
2042            float64
2043            float64
2044            float64
2045            float64
2046            float64
2047            float64
2048            float64
2049            float64
2050            float64
Sum             float64
scenario         object
dtype: object

In [54]:
# Convert columns with names from 2025 to 2050 to float
year_columns = [str(year) for year in range(2025, 2051)]
df[year_columns] = df[year_columns].astype(np.float32)

In [55]:
df.dtypes

Lookup.Value     object
Coben            object
2025            float32
2026            float32
2027            float32
2028            float32
2029            float32
2030            float32
2031            float32
2032            float32
2033            float32
2034            float32
2035            float32
2036            float32
2037            float32
2038            float32
2039            float32
2040            float32
2041            float32
2042            float32
2043            float32
2044            float32
2045            float32
2046            float32
2047            float32
2048            float32
2049            float32
2050            float32
Sum             float64
scenario         object
dtype: object

In [56]:
# Create total column
# df["total (£m)"] = df[ [ f'{i} (£m)' for i in range(2025, 2051)]].sum(axis=1)
df["total (£m)"] = df[ [ f'{i}' for i in range(2025, 2051)]].sum(axis=1)

In [57]:
# Rename columns so it does not contain special characters (not needed anymore I think)
df.columns = df.columns.str.replace(' (£m)', '')

In [58]:
# Rename columns: replace spaces and points with underscores
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '_')

In [59]:
df.head()

,Lookup_Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,2044,2045,2046,2047,2048,2049,2050,Sum,scenario,total
0,E01000001,Air quality,0.003375,0.003706,0.006998,0.008777,0.010981,0.014253,0.017711,0.022410,...,0.076150,0.078284,0.079530,0.079614,0.079098,0.078548,0.078011,1.216085,BNZ,1.216085
1,E01000001,Congestion,0.005570,0.009543,0.011914,0.013247,0.013655,0.014798,0.011850,0.008587,...,-0.034378,-0.036939,-0.040992,-0.042819,-0.044712,-0.046119,-0.048975,-0.369011,BNZ,-0.369011
2,E01000001,Dampness,0.000000,0.000000,0.000032,0.000032,0.000032,0.000006,0.000004,0.000005,...,0.000085,0.000082,0.000082,0.000081,0.000081,0.000080,0.000080,0.001297,BNZ,0.001297
3,E01000001,Diet change,0.000000,0.000000,0.006080,0.006109,0.006064,0.005997,0.005924,0.005848,...,0.004982,0.004915,0.004849,0.004783,0.004717,0.004652,0.004587,0.128954,BNZ,0.128954
4,E01000001,Excess cold,0.000000,0.000000,0.000002,0.000038,0.000044,0.000055,0.000064,0.000236,...,0.001895,0.001887,0.001859,0.001861,0.002016,0.001987,0.001984,0.024851,BNZ,0.024851


In [60]:
df.total

0         1.216085
1        -0.369011
2         0.001297
3         0.128954
4         0.024851
            ...   
557107    0.000000
557108    2.544464
557109    0.000000
557110    0.000000
557111    3.382245
Name: total, Length: 557112, dtype: float32

In [61]:
np.max(df.total)

69.5999984741211

In [62]:
# df[df.total == np.max(df.total)]

In [63]:
np.mean(df.total)

0.51890457

In [64]:
df.dtypes

Lookup_Value     object
Coben            object
2025            float32
2026            float32
2027            float32
2028            float32
2029            float32
2030            float32
2031            float32
2032            float32
2033            float32
2034            float32
2035            float32
2036            float32
2037            float32
2038            float32
2039            float32
2040            float32
2041            float32
2042            float32
2043            float32
2044            float32
2045            float32
2046            float32
2047            float32
2048            float32
2049            float32
2050            float32
Sum             float64
scenario         object
total           float32
dtype: object

## Join SE factors and cobenefs

In [65]:
df = pd.merge(df, df_socio, left_on='Lookup_Value', right_on='LSOA_DZ_CD', how='left')

In [66]:
df.head()

,Lookup_Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,Rurality,House_value,Fuel_Type,Fuel_consumption_total,Floor_area,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm
0,E01000001,Air quality,0.003375,0.003706,0.006998,0.008777,0.010981,0.014253,0.017711,0.022410,...,1.0,865000.0,1,0.0,68.0,0,0,40.241662,550.790202,365.079144
1,E01000001,Congestion,0.005570,0.009543,0.011914,0.013247,0.013655,0.014798,0.011850,0.008587,...,1.0,865000.0,1,0.0,68.0,0,0,40.241662,550.790202,365.079144
2,E01000001,Dampness,0.000000,0.000000,0.000032,0.000032,0.000032,0.000006,0.000004,0.000005,...,1.0,865000.0,1,0.0,68.0,0,0,40.241662,550.790202,365.079144
3,E01000001,Diet change,0.000000,0.000000,0.006080,0.006109,0.006064,0.005997,0.005924,0.005848,...,1.0,865000.0,1,0.0,68.0,0,0,40.241662,550.790202,365.079144
4,E01000001,Excess cold,0.000000,0.000000,0.000002,0.000038,0.000044,0.000055,0.000064,0.000236,...,1.0,865000.0,1,0.0,68.0,0,0,40.241662,550.790202,365.079144


In [67]:
# Rename cobenef to always keep the same name (changed depending the version)
df.rename(columns={'Coben': 'co_benefit_type'}, inplace=True)

## Aggregating the time

In [68]:
# Number of years merging
time_step = 5

In [69]:
years = list(range( 2025, 2051 ))
#years

In [70]:
len(years)

26

In [71]:
df[["2025", "2026"]]

,2025,2026
0,0.003375,0.003706
1,0.005570,0.009543
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
557107,0.000000,0.000000
557108,0.031351,0.044887
557109,0.000000,0.000000
557110,0.000000,0.000000


In [72]:
#  AGGREGATE TIME: can disable
if True:
    for i in range(0, len(years) - ( time_step - 1), time_step):
        window_years = [str(year) for year in years[i:i+5]]
        print(window_years)
        window_sum = df[window_years].sum(axis=1)
        df[f'Y{window_years[0]}_{window_years[-1]}'] = window_sum
        # df[f'{window_years[0]}_{window_years[-1]}'] = window_sum

    # Delete single values columns for space
    df = df.drop(columns=[str(year) for year in years])

['2025', '2026', '2027', '2028', '2029']
['2030', '2031', '2032', '2033', '2034']
['2035', '2036', '2037', '2038', '2039']
['2040', '2041', '2042', '2043', '2044']
['2045', '2046', '2047', '2048', '2049']


In [73]:
# df[ ['2025', '2026', '2027', '2028', '2029', 'Y2025_2029'] ]

In [74]:
# CONVERT INT64 into int32
df[df.select_dtypes(np.int64).columns] = df.select_dtypes(np.int64).astype(np.int32)

In [75]:
df

,Lookup_Value,co_benefit_type,Sum,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
0,E01000001,Air quality,1.216085,BNZ,1.216085,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.033836,0.114728,0.244001,0.350434,0.395074
1,E01000001,Congestion,-0.369011,BNZ,-0.369011,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.053929,0.041763,-0.057243,-0.146903,-0.211581
2,E01000001,Dampness,0.001297,BNZ,0.001297,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.000096,0.000069,0.000247,0.000399,0.000407
3,E01000001,Diet change,0.128954,BNZ,0.128954,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.018253,0.029237,0.027368,0.025592,0.023915
4,E01000001,Excess cold,0.024851,BNZ,0.024851,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.000085,0.001062,0.004247,0.007862,0.009611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557107,W01002040,Noise,0.000000,BNZ,0.000000,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.000000,0.000000,0.000000,0.000000,0.000000
557108,W01002040,Physical activity,2.544464,BNZ,2.544464,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.267307,0.433122,0.511427,0.576096,0.626393
557109,W01002040,Road repairs,0.000000,BNZ,0.000000,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.000000,0.000000,0.000000,0.000000,0.000000
557110,W01002040,Road safety,0.000000,BNZ,0.000000,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.000000,0.000000,0.000000,0.000000,0.000000


In [76]:
set(df.EPC)

{1, 2, 3, 4, 5, 6, 7, <NA>}

In [77]:
df.dtypes

Lookup_Value                      object
co_benefit_type                   object
Sum                              float64
scenario                          object
total                            float32
Nation                    string[python]
Region                    string[python]
LAD                       string[python]
LSOA_DZ_CD                string[python]
LSOA_DZ_NM                string[python]
Households                         int32
Population                         int32
Under_35                         Float64
Over_65                          Float64
EPC                                Int16
Median_Income                    Float64
Tenure                             int32
Typology                           int32
Unemployment                     Float64
Rurality                         Float64
House_value                      Float64
Fuel_Type                          int32
Fuel_consumption_total           Float64
Floor_area                       Float64
Gas_flag        

### Show max values (some values seem very skewed)

In [78]:
# Sort by cb
df_sorted = df.sort_values(by='total', ascending=False)
print(df_sorted.head())

       Lookup_Value    co_benefit_type        Sum scenario      total  \
556676    W01002002  Physical activity  69.600000      BNZ  69.599998   
167663    E01014714              Total  45.995239      BNZ  45.995239   
2891      E01000253              Total  45.359165      BNZ  45.359169   
271643    E01023839              Total  45.348804      BNZ  45.348808   
351251    E01030831              Total  44.933323      BNZ  44.933327   

         Nation           Region        LAD LSOA_DZ_CD LSOA_DZ_NM  ...  \
556676    Wales            Wales  W06000008  W01002002          -  ...   
167663  England       South West  E06000023  E01014714          -  ...   
2891    England           London  E09000003  E01000253          -  ...   
271643  England  Eastern England  E07000102  E01023839          -  ...   
351251  England       South East  E07000215  E01030831          -  ...   

        Gas_flag  Number_cars  Urban_trips    Total_vkm    Urban_vkm  \
556676         0            2   214.752476  

In [79]:
df[(df["co_benefit_type"] == "Noise") & (df["Lookup_Value"] == "N20000001")]

,Lookup_Value,co_benefit_type,Sum,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
405067,N20000001,Noise,0.289893,BNZ,0.289893,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.004885,0.024514,0.059251,0.099551,0.092327


In [80]:
len(set(df.LAD))

399

## Export table as parquet file

In [81]:
df.to_parquet('static/database.parquet')

In [82]:
# Not needed anymore, everything is in the same table
# df_socio.to_parquet('static/tableSocio.parquet')

In [83]:
df.columns

Index(['Lookup_Value', 'co_benefit_type', 'Sum', 'scenario', 'total', 'Nation',
       'Region', 'LAD', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Households', 'Population',
       'Under_35', 'Over_65', 'EPC', 'Median_Income', 'Tenure', 'Typology',
       'Unemployment', 'Rurality', 'House_value', 'Fuel_Type',
       'Fuel_consumption_total', 'Floor_area', 'Gas_flag', 'Number_cars',
       'Urban_trips', 'Total_vkm', 'Urban_vkm', 'Y2025_2029', 'Y2030_2034',
       'Y2035_2039', 'Y2040_2044', 'Y2045_2049'],
      dtype='object')

In [84]:
set(df.Nation)

{'England', 'NI', 'Scotland', 'Wales'}

In [85]:
df.co_benefit_type

0               Air quality
1                Congestion
2                  Dampness
3               Diet change
4               Excess cold
                ...        
557107                Noise
557108    Physical activity
557109         Road repairs
557110          Road safety
557111                Total
Name: co_benefit_type, Length: 557112, dtype: object

## Export subset for faster development 

In [86]:
df

,Lookup_Value,co_benefit_type,Sum,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
0,E01000001,Air quality,1.216085,BNZ,1.216085,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.033836,0.114728,0.244001,0.350434,0.395074
1,E01000001,Congestion,-0.369011,BNZ,-0.369011,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.053929,0.041763,-0.057243,-0.146903,-0.211581
2,E01000001,Dampness,0.001297,BNZ,0.001297,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.000096,0.000069,0.000247,0.000399,0.000407
3,E01000001,Diet change,0.128954,BNZ,0.128954,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.018253,0.029237,0.027368,0.025592,0.023915
4,E01000001,Excess cold,0.024851,BNZ,0.024851,England,London,E09000001,E01000001,-,...,0,0,40.241662,550.790202,365.079144,0.000085,0.001062,0.004247,0.007862,0.009611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557107,W01002040,Noise,0.000000,BNZ,0.000000,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.000000,0.000000,0.000000,0.000000,0.000000
557108,W01002040,Physical activity,2.544464,BNZ,2.544464,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.267307,0.433122,0.511427,0.576096,0.626393
557109,W01002040,Road repairs,0.000000,BNZ,0.000000,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.000000,0.000000,0.000000,0.000000,0.000000
557110,W01002040,Road safety,0.000000,BNZ,0.000000,Wales,Wales,W06000014,W01002040,-,...,1,1,1006.99169,15293.09387,10136.69016,0.000000,0.000000,0.000000,0.000000,0.000000


In [87]:
df.columns

Index(['Lookup_Value', 'co_benefit_type', 'Sum', 'scenario', 'total', 'Nation',
       'Region', 'LAD', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Households', 'Population',
       'Under_35', 'Over_65', 'EPC', 'Median_Income', 'Tenure', 'Typology',
       'Unemployment', 'Rurality', 'House_value', 'Fuel_Type',
       'Fuel_consumption_total', 'Floor_area', 'Gas_flag', 'Number_cars',
       'Urban_trips', 'Total_vkm', 'Urban_vkm', 'Y2025_2029', 'Y2030_2034',
       'Y2035_2039', 'Y2040_2044', 'Y2045_2049'],
      dtype='object')

In [88]:
# Assuming df is your DataFrame and 'column_name' is the column you're interested in
# df_sampled = df.groupby('LAD').apply(lambda x: x.sample(n=10)).reset_index(drop=True)
df_sampled = df[df.Nation == "NI"]

In [89]:
df_sampled

,Lookup_Value,co_benefit_type,Sum,scenario,total,Nation,Region,LAD,LSOA_DZ_CD,LSOA_DZ_NM,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
405060,N20000001,Air quality,0.553835,BNZ,0.553835,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.030565,0.060087,0.109074,0.151467,0.169310
405061,N20000001,Congestion,-0.038290,BNZ,-0.038290,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.115207,0.153278,0.041410,-0.090639,-0.198814
405062,N20000001,Dampness,0.016224,BNZ,0.016224,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.000167,0.001805,0.003419,0.004575,0.005196
405063,N20000001,Diet change,0.055298,BNZ,0.055298,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.007827,0.012538,0.011736,0.010975,0.010255
405064,N20000001,Excess cold,0.049890,BNZ,0.049890,NI,Northern Ireland,N09000001,N20000001,Dunsilly_A1,...,0,2,414.15324,12922.63505,2584.316217,0.000935,0.003723,0.009302,0.014391,0.017935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450391,N20003780,Noise,0.021659,BNZ,0.021659,NI,Northern Ireland,N09000011,N20003780,Ards_Peninsula_L4,...,0,2,395.287202,12333.96665,2466.592139,0.004047,0.001283,0.001808,0.007354,0.006864
450392,N20003780,Physical activity,0.872864,BNZ,0.872864,NI,Northern Ireland,N09000011,N20003780,Ards_Peninsula_L4,...,0,2,395.287202,12333.96665,2466.592139,0.091624,0.148513,0.175385,0.197610,0.215018
450393,N20003780,Road repairs,-0.003145,BNZ,-0.003144,NI,Northern Ireland,N09000011,N20003780,Ards_Peninsula_L4,...,0,2,395.287202,12333.96665,2466.592139,0.000791,0.000804,-0.000307,-0.001461,-0.002355
450394,N20003780,Road safety,-0.022750,BNZ,-0.022750,NI,Northern Ireland,N09000011,N20003780,Ards_Peninsula_L4,...,0,2,395.287202,12333.96665,2466.592139,0.022958,0.030802,0.007951,-0.020860,-0.048901


In [90]:
# df_sampled[(df_sampled["Lookup_Value"] == "N20000001") & (df_sampled["co_benefit_type"] == "Dampness")][["2028", "scenario"]]

KeyError: "['2028'] not in index"

In [91]:
df_sampled.to_parquet('static/database_onlyIreland.parquet')